# The Main Pipline for Task 2 (Team 72)

### Part 1: Blip inference of tweet and other info such as images and videos to generate prompts

In [1]:
! pip install -i https://test.pypi.org/simple/ bitsandbytes
! pip install transformers==4.30
! pip install -i https://test.pypi.org/simple/ bitsandbytes
! pip install accelerate

Looking in indexes: https://test.pypi.org/simple/
  Obtaining dependency information for bitsandbytes from https://test-files.pythonhosted.org/packages/5c/e0/597d593ec3b6cf5ea7eb4894a545045bd95611de8a316a2a1eaa838a2459/bitsandbytes-0.39.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 13.3 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for transformers==4.30 from https://files.pythonhosted.org/packages/e2/72/1af3d38e98fdcceb3876de4567ac395a66c26976e259fe2d46266e052d61/transformers-4.30.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0


In [2]:
import os
import re
import cv2
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from io import BytesIO
from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from transformers import (
    Blip2Processor,
    Blip2ForConditionalGeneration,
    Blip2Model,
    Blip2VisionModel,
    Blip2QFormerModel,
    Blip2QFormerConfig,
    OPTConfig,
    Blip2Config,
    BitsAndBytesConfig,
)
import torch
import argparse
import PIL

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Blip2Processor.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    load_in_8bit=True,
    device_map="auto",
    max_memory={0: "10GB"},
    torch_dtype=torch.float16,
    use_safetensors=True,
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
)

blip2_model = (
    Blip2ForConditionalGeneration.from_pretrained(  # For better performance on Images
        "Salesforce/blip2-opt-2.7b",
        device_map="auto",
        max_memory={0: "10GB"},
        quantization_config=quantization_config,
        force_download=False,
    )
)


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.50G [00:00<?, ?B/s]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
! git clone https://github.com/salesforce/BLIP
! ls /kaggle/working/BLIP/models
! pip install flash-attn --no-build-isolation
! pip install timm==0.4.12 fairscale==0.4.4

Cloning into 'BLIP'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 277 (delta 137), reused 136 (delta 135), pack-reused 112
Receiving objects: 100% (277/277), 7.03 MiB | 22.58 MiB/s, done.
Resolving deltas: 100% (152/152), done.
__init__.py  blip_itm.py   blip_pretrain.py   blip_vqa.py  nlvr_encoder.py
blip.py      blip_nlvr.py  blip_retrieval.py  med.py	   vit.py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.3.6-cp310-cp310-linux_x86_64.whl size=56561532 sha256=0552c92bb12d5

In [5]:
%cd BLIP
from models.blip import blip_decoder

# For performance on Videos we use Blip1 to get low instance time and high performance
image_size = 384
blip1_model_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth"
blip1_model = blip_decoder(
    pretrained=blip1_model_url, image_size=image_size, vit="base"
)
blip1_model.eval()
blip1_model = blip1_model.to(device)

/kaggle/working/BLIP


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.97G/1.97G [00:10<00:00, 199MB/s] 


reshape position embedding from 196 to 576
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth


In [6]:
def image_inference(twitter_data):
    image_pattern = re.compile(r'https://pbs.twimg.com/media/[^\s]+?format=jpg&name=small')

    image_matches = re.findall(image_pattern, twitter_data)
    
    image_captions = ''
    
    for image_url in image_matches:
        try:
            response = requests.get(image_url)
            try:
                image = Image.open(BytesIO(response.content))
            except PIL.UnidentifiedImageError:
                return None

            # The output of 4bit precision seems good and the GPU memory required for the setup is
            # around 10GB with CPU memory max around 12-13 GB
            # The torch data type used here -> torch.float16
            with torch.backends.cuda.sdp_kernel(
                enable_flash=True, enable_math=False, enable_mem_efficient=False
            ):
                inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

                ### The above steps are done to reduce and get just the outputs of qformers
                generated_ids = blip2_model.generate(**inputs)
                generated_text = processor.batch_decode(
                    generated_ids, skip_special_tokens=True
                )[0].strip()
                
                image_captions = image_captions + " " + generated_text + ":"
        
        except requests.exceptions.MissingSchema:
             image_captions = image_captions

    return image_captions


In [7]:

def load_demo_image(image_size, device, pil_image):
    transform = transforms.Compose(
        [
            transforms.Resize(
                (image_size, image_size), interpolation=InterpolationMode.BICUBIC
            ),
            transforms.ToTensor(),
            transforms.Normalize(
                (0.48145466, 0.4578275, 0.40821073),
                (0.26862954, 0.26130258, 0.27577711),
            ),
        ]
    )
    image = transform(pil_image).unsqueeze(0).to(device)
    return image


def process_video(input_twitter_media, model, device):
    url_pattern = re.compile(r"https?://\S+")

    matches = url_pattern.findall(input_twitter_media)
    
    for m in matches:
        # print(m.find('mp4'))
        if m.find("mp4") != -1:
            video_url = m
            break
    try:
        response = requests.get(video_url, stream=True)

        video_filename = "downloaded_gif.mp4"
        with open(video_filename, "wb") as video_file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    video_file.write(chunk)
    except requests.exceptions.ContentDecodingError:
        return "", ""
    cap = cv2.VideoCapture(video_filename)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_indices = np.linspace(0, total_frames - 1, 10, dtype=int)

    frames = []
    for idx in sample_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    video_captions = ""

    for idx, frame in enumerate(frames):
        pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        image = load_demo_image(
            image_size=image_size, device=device, pil_image=pil_frame
        )

        caption = model.generate(
            image, sample=False, num_beams=1, max_length=20, min_length=5
        )
        video_captions = video_captions + " " + caption[0] + ":"

    return video_url, video_captions


In [8]:

def process_gif(input_twitter_media, model, device):

    jpg_pattern = re.compile(r'thumbnailUrl=\'(https://pbs.twimg.com/tweet_video_thumb/[^\']+)')
    mp4_pattern = re.compile(r'video/mp4\', url=\'(https://video.twimg.com/tweet_video/[^\']+)')

    image_urls = re.findall(jpg_pattern, input_twitter_media)[0]
    gif_url = re.findall(mp4_pattern, input_twitter_media)[0]

    
    gif_filename = "downloaded_gif.mp4"
    gif_captions = ""
    
    gif_output = False 
    
    if gif_url != "":
        try:
            with open(gif_filename, "wb") as gif_file:
                response = requests.get(gif_url, stream=True)
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        gif_file.write(chunk)
            gif_output = True
        except requests.exceptions.ContentDecodingError:
            gif_captions = ""
        
        if gif_output:
                cap = cv2.VideoCapture(gif_filename)
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                sample_indices = np.linspace(0, total_frames - 1, 4, dtype=int)

                frames = []
                for idx in sample_indices:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frames.append(frame)
                
                cap.release()
       
                for idx, frame in enumerate(frames):
                    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    image = load_demo_image(
                        image_size=image_size, device=device, pil_image=pil_frame
                    )

                    caption = model.generate(
                        image, sample=False, num_beams=1, max_length=20, min_length=5
                    )
                    gif_captions = gif_captions + " " + caption[0] + ":"
    
    image_captions = ""
    
    if not gif_output:
        for image_url in image_urls:
            generated_text = image_inference(image_url[:-2])
            image_captions = str(image_captions) + str(generated_text)
            
    gif_captions = str(gif_captions) + str(image_captions)
    return gif_url, gif_captions

In [9]:
userdata = pd.read_csv('/kaggle/input/input-params-and-embeddings/complete_username_data.csv')

def company_username_inference(username):
    """Use data obtained from Twitter API and wikidata to do inference."""
    user = userdata[userdata["username"] == username]
    if userdata is not None:
        return user
    else:
        return None

### Part 2: 

In [33]:
from typing import Any

columns_dict ={
    'likes': 'Expected Likes',
    'username': 'Username',
    'inferred company': 'Company Name',
    'date': 'Date Time',
    'content': 'Tweet Content'
}

def process_inputs(
    input_twitter_media: str,
    expected_likes: Any,
    inferred_company: str,
    twitter_username: str,
    date: str
):
    data_item = {}

    if "Photo" in input_twitter_media:
        generated_text = image_inference(input_twitter_media)
        data_item["content"] = [generated_text] + data_item.get("content", [])

    elif "Video" in input_twitter_media:
        video_url, video_captions = process_video(
            input_twitter_media, blip1_model, device
        )
        data_item["content"] = [video_captions] + data_item.get("content", [])
    
    elif "Gif" in input_twitter_media:
        print('gif')
        gif_url, gif_captions = process_gif(
            input_twitter_media, blip1_model, device
        )
        data_item["content"] = [gif_captions] + data_item.get("content", [])
        
    if twitter_username != '':     
        twitter_data = company_username_inference(twitter_username)     
        if len(twitter_data) != 0:
            if twitter_data['location'].item() != "":
                data_item["content"].append(f"Location: {twitter_data['location'].item()}")
                data_item["content"].append(f"Description: {twitter_data['description'].item()}")
    
    if expected_likes is not None:
        data_item["content"].append(f"Expected Likes: {expected_likes}")
    if inferred_company != '':
        data_item["content"].append(f"inferred company: {inferred_company}")
    if date != '':
        data_item["content"].append(f"date: {date}")

    data_item["content"] = [item for item in data_item["content"] if item is not None]

    data_item["content"] = ", ".join(data_item["content"])
    
    data_item[
        "instruction"
    ] = "You are a content generator, you have to generate the tweet content based on the expected likes and other additional information."
    data_item['output'] = ''
    data_item['input'] = data_item['content']
    return data_item

## Part 2: Inference Using LLAMA finetuning approaches

#### (A) Incontext Learning

In [11]:
!pip install -q peft==0.4.0 trl==0.4.7 sentencepiece
!pip install faiss-cpu
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=23f2403530ad8e4b0a0accf75ab12d135604533bf28f8db76360892925a48dd5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret('HF')
!huggingface-cli login --token $secret_value_0 --add-to-git-credential

Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
import gc
import logging
from time import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import ctypes
import json
import csv

import faiss
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)

def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

# Load SBERT model
model_sbert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [14]:
import pickle
with open('/kaggle/input/input-params-and-embeddings/embeddings_200000_new.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    train_tweet_data = stored_data['input']
    train_embeddings = stored_data['embeddings']

In [15]:
# Build an index with Faiss
dimension = len(train_embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(train_embeddings)

# Function to perform similarity search
def search(query, k=5):
    query_embedding = model_sbert.encode([query])[0]
    _, result_indices = index.search(query_embedding.reshape(1, -1), k)
    return result_indices[0]

In [16]:
incontext_llama = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype=torch.bfloat16,load_in_4bit=True, device_map="cuda:0")
incontext_llama_tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

In [17]:
with open('/kaggle/input/input-params-and-embeddings/io_embeddings_200000_new.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    input_text = stored_data['input']
    output_text = stored_data['output']

# Initialize the pipeline
pipe = pipeline(task="text-generation", model=incontext_llama, tokenizer=incontext_llama_tokenizer, max_length=1024)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
def incontext_inference(input_data: str):
    indexes = search(input_data)
    id1 = indexes[0]
    id2 = indexes[1]
    id3 = indexes[2]
    id4 = indexes[3]
    print('Nearest Neighbours: ')
    print(input_text[id1])
    print(input_text[id2])
    print(input_text[id3])
    print(input_text[id4])
    
    
    prompt = f'''<s>[INST] <<SYS>>
        You are a content generator, you have to generate the tweet text based on the expected likes and other additional information provided to you. Write the tweet text and nothing else.
        <</SYS>>

        {input_text[id1]} [/INST] { output_text[id1] } </s><s>{input_text[id2]} [/INST] { output_text[id2] } </s><s>{input_text[id3]} [/INST] { output_text[id3] } </s><s>[INST] {input_data} [/INST]'''

    output = pipe(prompt)
    return output[0]["generated_text"].split('[/INST]')[-1].split('tweet text: ')[-1]

#### (B) LLAMA Adapter

In [19]:
pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
! cp -r /kaggle/input/adobe-inter-iit/adobe_behaviour_simulation/task2/llama_adapter /kaggle/working/

In [21]:
cd /kaggle/working/llama_adapter

/kaggle/working/llama_adapter


In [22]:
! pip install -r requirements.txt
! pip install -e

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=f0fe0dfc0a92a8663313a676795d5129b56e504fe9b3e541468c23077f5ca890
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [23]:
import re
import json

def get_response(response):
    return response.split("### Response:")[1][1:]

def remove(text):
    cleaned_text = re.sub(r'<\/?hyperlink[^>]*>|<\/?mention[^>]*>', '', text)
    return cleaned_text

def find_repeated_word(text, threshold=5):
    words = text.split()
    word_count = {}

    for word in words:
        word_count[word.lower()] = word_count.get(word.lower(), 0) + 1

    repeated_words = [word for word, count in word_count.items() if count > threshold]
    return repeated_words

def remove_excess_occurrences(text):
    repeated_words = find_repeated_word(text)
    words = text.split()
    for word in repeated_words:
        count = 0
        for i in range(len(words)):
            if words[i].lower() == word.lower():
                count += 1
                if count > 5:
                    words[i] = ''  # Replace the word after reaching the maximum occurrences

    return ' '.join(words)

def remove_mention(text):
    if isinstance(text, str):  # Check if the value is a string
        return text.split("<mention")[0]
    else:
        return text 

def remove_expected(text):
    if isinstance(text, str):  # Check if the value is a string
        text = text.split("Expected likes")[0]
        text = text.split("expected likes")[0]
        text = text.split("Expected Likes")[0]
        return text
    else:
        return text

def remove_hashtag(text):
    if isinstance(text, str):
        return " #".join(text.split("#")[:5])
    else:
        return text

def remove_a(text):
    if isinstance(text, str):
        return " @".join(text.split("@")[:5])
    else:
        return text

def remove_amp(text):
    if isinstance(text, str):
        return "amp".join(text.split("amp")[:5])
    else:
        return text

# Load a single item from the JSON file
def get_llama_adapter_output(filename):
    json_data = json.load(open(filename))
    single_item = json_data[0]  # Assuming you want to process the first item

    # Apply cleaning functions
    content = single_item["content"]
    content = get_response(content)
    content = remove(content)
    content = remove_excess_occurrences(content)
    content = remove_mention(content)
    content = remove_expected(content)
    content = remove_hashtag(content)
    content = remove_a(content)
    content = remove_amp(content)
    return content

## Inference

In [24]:
input_twitter_media = "[Video(thumbnailUrl='https://pbs.twimg.com/media/EIsfIGNWoAEPmAu.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1192072750581633024/vid/320x400/5Y3qWndCSlQx24M_.mp4?tag=13', bitrate=632000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1192072750581633024/pl/ZMhUCt2SG1uO71jP.m3u8?tag=13', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1192072750581633024/vid/360x450/vQTEz9WbxhA-nPde.mp4?tag=13', bitrate=832000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1192072750581633024/vid/720x900/s01hwkz1PTj3yfjj.mp4?tag=13', bitrate=2176000)], duration=203.64, views=1077722)]"
expected_likes = None
inferred_company = 'liverpool'
twitter_username = 'LFC' 
date = '2019-11-09 08:15:00'

#### Inference Caption By Blip

In [34]:
import time

import warnings
warnings.filterwarnings("ignore")

start_time = time.time()

output = process_inputs(input_twitter_media,
    expected_likes,
    inferred_company,
    twitter_username,
    date)

end_time = time.time()

elapsed_time = end_time - start_time
print(f'Blip Captions Output: {output}')
print("Elapsed time: ", elapsed_time) 

Blip Captions Output: {'content': ' a soccer player in action: a soccer game with a ball in the middle: a soccer player with a ball: a soccer match with a player on the field: a soccer game with a player kicking the ball: a soccer player with his arm raised in the air: a man holding a soccer ball: a soccer game with the goal: a soccer match with a player on the field: a red television logo:, inferred company: liverpool, date: 2019-11-09 08:15:00', 'instruction': 'You are a content generator, you have to generate the tweet content based on the expected likes and other additional information.', 'output': '', 'input': ' a soccer player in action: a soccer game with a ball in the middle: a soccer player with a ball: a soccer match with a player on the field: a soccer game with a player kicking the ball: a soccer player with his arm raised in the air: a man holding a soccer ball: a soccer game with the goal: a soccer match with a player on the field: a red television logo:, inferred company

In [35]:
results = []
results.append(output)
import json
# Save the list of dictionaries to a JSON file
with open('test.json', 'w') as json_file:
    json.dump(results, json_file, indent=2)

In [36]:
!cat test.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[
  {
    "content": " a soccer player in action: a soccer game with a ball in the middle: a soccer player with a ball: a soccer match with a player on the field: a soccer game with a player kicking the ball: a soccer player with his arm raised in the air: a man holding a soccer ball: a soccer game with the goal: a soccer match with a player on the field: a red television logo:, inferred company: liverpool, date: 2019-11-09 08:15:00",
    "instruction": "You are a content generator, you have to generate the tweet content based on the expected likes and other additional information.",
    "output": "",
    "input": " a soccer player in action: a soccer game with a ball

#### Output Processing with Incontext Learning

In [37]:

start_time = time.time()

generated_caption = incontext_inference(output)
print(generated_caption)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time) 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Nearest Neighbours: 
[video description a soccer game with a player on the field: a soccer game with a player on the field: a soccer game with a ball in the middle of the field: a soccer game with a ball in the middle of the field: a soccer match with a crowd watching: a
[video description a soccer game with a ball in the middle of the field: a soccer field with a ball in the middle: a soccer game with a goal: a soccer game with a goal: a soccer game: a soccer player with a red jersey: two men in a soccer game: a soc
[video description a soccer game with a yellow background: a soccer game with a player on the field: a soccer game with a goal: a soccer game with a player in the middle: a soccer player with his arms outstretched: a soccer player celebrating: a socc
[video description a soccer game with a player on the field: a soccer game with a ball in the middle: a soccer game with a ball in the middle: a soccer game with a player on the field: a soccer player with his head in the air:

#### Output processing with LLAMA-Adapter 

In [38]:
! CUDA_VISIBLE_DEVICES=1 bash example_new.sh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
1
[PosixPath('alpaca_finetuning_v1/llama/weights_7b/7B/consolidated.00.pth')]
Loading
Transformer(
  (tok_embeddings): ParallelEmbedding()
  (layers): ModuleList(
    (0-31): 32 x TransformerBlock(
      (attention): Attention(
        (wq): ColumnParallelLinear()
        (wk): ColumnParallelLinear()
        (wv): C

In [39]:
!cat company_time.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[{"content":"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a content generator, you have to generate the tweet content based on the expected likes and other additional information.\n\n### Input:\n a soccer player in action: a soccer game with a ball in the middle: a soccer player with a ball: a soccer match with a player on the field: a soccer game with a player kicking the ball: a soccer player with his arm raised in the air: a man holding a soccer ball: a soccer game with the goal: a soccer match with a player on the field: a r

In [40]:
get_llama_adapter_output('company_time.json')

"Liverpool's Mohamed Salah has scored 10 goals in 10 games this season.  #soccer  #football  #sports  #soccerplayer "